### Task1
To complete this task, use the data set in the attached file. Indicate the answer to each of the
following steps and time to complete the entire task.


1.1. Download the data set movie_metadata.csv, which contains data about films from IMDb
(Internet Movie Database).

[movie_metadata.csv](https://s3-us-west-2.amazonaws.com/secure.notion-static.com/ad88c1ae-a16a-4a9b-a0dd-a12ecc5a9568/movie_metadata.csv)

In [4]:
import pandas as pd
df = pd.read_csv('/home/alex/HDD/workshop/.datasets/yndx/movie_metadata.csv')
df.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0$,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0$,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0$,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0$,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,0.0$,NaN,12.0,7.1,NaN,0


1.2. The __duration__ column contains data on the film __length__. How many missing values are there
in this column?

In [3]:
df.duration.isnull().sum()

15

1.3. Replace the missing values in the duration column with the median value for this column.

In [4]:
duration_m = df.duration.median()
df.duration.fillna(duration_m,inplace=True)
df.duration.isnull().sum()

0

1.4. What is the average film length? Give the answer as a floating-point figure rounded to
two decimal places.

In [5]:
round(df.duration.mean(),2)

107.19

1.5. Create a movie_duration_category column, which will contain three categories
depending on the film length:
• Category "1. <90" if the film is less than 90 minutes long
• Category "2. 90–120" if the film is between 90 minutes and two hours long (inclusively)
• Category "3. >120" if the film is more than two hours long

In [10]:
short = 90
long = 120
labels = [f'<{short}', f'{short}-{long}',f'>{long}']
df['movie_duration_category'] = pd.cut(df['duration'],bins = (df.duration.min(),short,long ,df.duration.max()), labels = labels,include_lowest = True)
df.loc[:,('duration','movie_duration_category')]

,duration,movie_duration_category
0,178.0,>120
1,169.0,>120
2,148.0,>120
3,164.0,>120
4,103.0,90-120
...,...,...
5038,87.0,<90
5039,43.0,<90
5040,76.0,<90
5041,100.0,90-120


1.6. Build a summary table for films released after 2000 (inclusively), to list the numbers of
films:<br>
• Table rows: year<br>
• Table columns: movie duration category ("<90", "90–120", ">120")<br>
• The year of release should be displayed in the YYYY format.<br>

In [11]:
def summary(df):
    df = df[df.title_year>=2000]
    df['title_year']  = df.title_year.astype(int)
    return df.groupby(df.title_year).movie_duration_category.value_counts().unstack()
summary(df)

/home/alex/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,<90,90-120,>120
title_year,,,
2000,33,104,34
2001,36,113,39
2002,40,142,27
2003,37,102,30
2004,38,134,42
2005,40,133,48
2006,50,136,53
2007,37,124,43
2008,42,147,36


1.7. How many films between 90 minutes and two hours long were released in 2008?

In [12]:
summary(df).loc[2008,'90-120']

/home/alex/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


147

1.8. The plot_keywords column holds keywords characterizing the film's plot. Using the data in this column, create a column called __movie_plot_category__, to contain four categories depending on the key words in the column: <br>
* Category "love_and_death" if the keywords include both "love" and "death" 
* Category "love" if the keywords include the word "love" 
* Category "death" if the keywords include the word "death" 
* Category "other" if the keywords do not meet the conditions above

In [34]:
df.plot_keywords.fillna('dummy', inplace = True)
df['movie_plot_category'] = 'other'
df.loc[df.plot_keywords.str.contains('love'),'movie_plot_category'] = 'love'
df.loc[df.plot_keywords.str.contains('death'),'movie_plot_category'] = 'death'
df.loc[df.plot_keywords.str.contains('death') * df.plot_keywords.str.contains('love'),'movie_plot_category'] = 'love_and_death'

1.9. The imdb_score column shows a viewer rating for the film. Build a table to reflect the average rating of films depending on which movie_plot_category category they belong to.

In [35]:
df.groupby(df.movie_plot_category).imdb_score.mean()

movie_plot_category
death             6.535465
love              6.580769
love_and_death    6.506250
other             6.431422
Name: imdb_score, dtype: float64

1.10. What is the average rating of films in the "love" category? Give the answer as a floatingpoint
figure rounded to two decimal places.

In [37]:
round(df.groupby(df.movie_plot_category).imdb_score.mean().love,2)

6.58

1.11. The budget column contains the film's budget. What is the median budget for all the films
listed? Give the answer as an integer.

In [10]:
df.budget = pd.to_numeric(df.budget.astype(str).str.replace('[$,]', ''), downcast = 'integer')
int(df.budget.median())

/home/alex/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


15000000

### TASK2
To complete this task, use the data set in the attached file. Indicate the answer to each of the
following steps and time to complete the entire task.

[event_data.csv](https://s3-us-west-2.amazonaws.com/secure.notion-static.com/3d249c69-fd8f-419f-8606-554c9a089cd7/event_data.csv)

1. Download the event_data.csv dataset, which contains data on the use of the mobile
application of users who registered from July 29 to September 1, 2019:
* user_id - user identifier;
* event_date - time of the event;
* event_type - type of event: registration - registration in the application; simple_event click event in the application; purchase - an event of purchase within the application;
* purchase_amount - purchase amount.

In [1]:
import pandas as pd
df2 = pd.read_csv('/home/alex/HDD/workshop/.datasets/yndx/event_data.csv', parse_dates = ['event_date'])
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79742 entries, 0 to 79741
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   user_id          79742 non-null  object        
 1   event_date       79742 non-null  datetime64[ns]
 2   event_type       79742 non-null  object        
 3   purchase_amount  6207 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 2.4+ MB


2. Highlight user cohorts based on the week of registration in the application. 
The cohort identifier should be the week ordinal (for example, the week from July 29 to August 4 should have identifier 31).

In [2]:
df2['event_week'] = df2['event_date'].dt.isocalendar().week
def split(df2):
    regs = df2.loc[df2.event_type == 'registration']
    events = df2.loc[df2.event_type != 'registration']
    events = events.assign(reg_week = 0)
    for _,user in regs.user_id.iteritems():
        events.reg_week.mask(events.user_id == user, int(regs.loc[regs.user_id == user].event_week), inplace = True)
    return events, regs

events = pd.read_csv('events_csv')
regs = df2.loc[df2.event_type == 'registration']
events

,Unnamed: 0,user_id,event_date,event_type,purchase_amount,event_week,reg_week
0,24,af679d,2019-07-29 02:25:19,simple_event,NaN,31,31
1,73,71ac11,2019-07-29 06:19:19,simple_event,NaN,31,31
2,78,85367a,2019-07-29 06:37:26,simple_event,NaN,31,31
3,79,9b887b,2019-07-29 06:39:29,simple_event,NaN,31,31
4,80,b2e16e,2019-07-29 06:51:51,simple_event,NaN,31,31
...,...,...,...,...,...,...,...
69741,79737,930c23,2019-09-01 23:57:41,simple_event,NaN,35,32
69742,79738,a84999,2019-09-01 23:57:50,simple_event,NaN,35,33
69743,79739,175e4d,2019-09-01 23:59:40,simple_event,NaN,35,32
69744,79740,1c2210,2019-09-01 23:59:51,simple_event,NaN,35,33


3. How many unique users in the cohort with ID 33?

In [3]:
len(events.loc[events.reg_week == 33].user_id.unique())

2045

4. For each event, highlight the indicator lifetime - the weekly lifetime of the cohort. <br>
The __lifetime indicator__ is calculated based on the serial number of the week in which the event
is committed, relative to the week of registration. For example, an event committed on
August 3 by a user from a cohort of registrants at 31 weeks will be committed on the zero
week of lifetime, and an event committed by the same user on August 5 will be committed
on the first week of lifetime).

In [4]:
events['lifetime'] = events.event_week - events.reg_week

5. Build a summary table of changes in the __Retention Rate__ for cohorts depending on lifetime.

RR is expressed as a percentage of a company’s existing customers who remain loyal within that time frame. 

In [14]:
# def cohort(df, lifetime = 0, reg_week = 31):
#     return df.loc[(df['reg_week']==reg_week) & (df['lifetime']==lifetime)]
# cohort(events)

,Unnamed: 0,user_id,event_date,event_type,purchase_amount,event_week,reg_week,lifetime
0,24,af679d,2019-07-29 02:25:19,simple_event,NaN,31,31,0
1,73,71ac11,2019-07-29 06:19:19,simple_event,NaN,31,31,0
2,78,85367a,2019-07-29 06:37:26,simple_event,NaN,31,31,0
3,79,9b887b,2019-07-29 06:39:29,simple_event,NaN,31,31,0
4,80,b2e16e,2019-07-29 06:51:51,simple_event,NaN,31,31,0
...,...,...,...,...,...,...,...,...
3778,5752,fce459,2019-08-04 23:53:36,simple_event,NaN,31,31,0
3779,5753,7e3a50,2019-08-04 23:54:53,simple_event,NaN,31,31,0
3780,5754,3b3712,2019-08-04 23:55:23,simple_event,NaN,31,31,0
3781,5756,5b7d54,2019-08-04 23:55:56,purchase,20.0,31,31,0


In [45]:
d = {}
for reg_week in range(31,36):
    d[reg_week] = ()
    for lifetime in range(5):
        d[reg_week] += events.loc[(events['reg_week']==reg_week) & (events['lifetime']==lifetime)].user_id.nunique(),

data =pd.DataFrame(d)
data = data / data.sum()
print('in %')
data = data.round(5)
data*100

in %


,31,32,33,34,35
0,27.326,28.872,33.113,45.514,100.0
1,32.656,34.098,38.993,54.486,0.0
2,22.157,23.778,27.894,0.000,0.0
3,12.567,13.252,0.000,0.000,0.0
4,5.294,0.000,0.000,0.000,0.0


6. What is the 3 week retention rate for a cohort with ID 32? Give the answer in percent,rounded to 2 decimal places, inclusive.

13.25%

7. Build a summary table of changes in the indicator ARPPU (Average Revenue Per Paying User) for cohorts depending on lifetime.

In [31]:
def arppu(cohort):
    total_revenue = cohort.purchase_amount.sum()
    paying_users = cohort.loc[cohort.purchase_amount.notna()].user_id.nunique()
    return total_revenue / paying_users
    
d = {}
for reg_week in range(31,36):
    d[reg_week] = ()
    for lifetime in range(5):
        cohort = events.loc[(events['reg_week']==reg_week) & (events['lifetime']==lifetime)]
        d[reg_week]+=(arppu(cohort)),

data = pd.DataFrame(d)
data

/home/alex/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  


,31,32,33,34,35
0,30.445205,32.978723,33.186441,33.843416,34.094955
1,35.413793,35.024958,35.858086,35.904936,NaN
2,35.911765,35.294118,34.784810,NaN,NaN
3,33.081081,33.297872,NaN,NaN,NaN
4,31.406250,NaN,NaN,NaN,NaN


8. What is the 3-week ARPPU of a cohort with ID 31? Give the answer with a floating point number, rounded to 2 decimal places, inclusive.

In [32]:
data.loc[3,31].round(2)

33.08

9. What is the median time between user registration and first purchase? Give the answer in seconds (!) As an integer.

In [33]:
purchases = events.loc[events.purchase_amount.notna()]
first_purchase = purchases.groupby('user_id').event_date.min()
registrations = regs.set_index(regs.user_id)
data = registrations.loc[first_purchase.index]

data['first_purchase'] = pd.to_datetime(first_purchase)
difference = data.event_date - data.first_purchase
difference.median().seconds

83626

### Task3 
How would you answer the student's question below? <br>
Your task is to get your message across in such a way that a beginner can understand your explanation. <br>
You can do this any way you want (pictures, GIFs, metaphors, anything) so long as it makes your explanation clear. <br>
Answer the question: _"What is the difference between DataFrame and Series?"_
Indicate how much time you spent completing this task.

The __DataFrame__ is the primary pandas data structure. It can be thought as dict-like container for __Series__ objects <br>
_Why DataFrame is a data structure and Series is an object?_ <br>
* because here we speak of DF abstractly, as a description of a general Class of objects; when we speak of objects, we usually assume that an instantiation procedure has been called, and the data structure was created in computer's memory. 

The __Series__ is the data structure for a single column of a DataFrame, not only conceptually, but literally, i.e. the data in a DataFrame is actually stored in memory as a collection of Series. <br>

DF contains 2 labeled axes, while Series only one.  


more on [DataFrames](https://pandas.pydata.org/pandas-docs/version/0.13.1/generated/pandas.DataFrame.html), [Series](https://pandas.pydata.org/pandas-docs/stable/reference/series.html) <br>

### Task4
You are given two random variables X and Y. <br>
E(X) = 0.5, Var(X) = 2 <br>
E(Y) = 7, Var(Y) = 3.5 <br>
cov (X, Y) = -0.8 <br>
Find the variance of the random variable Z = 2X - 3Y

In [12]:
import numpy as np
# x(E,D)?
x = np.array((0.5, 2))
y = np.array((7, 3.5))
covariance = -0.8
a = 2
b = -3
var_X, var_Y = [i[1] for i in (x,y) ]
variance = a**2*var_X + b**2*var_Y + 2*a*b* covariance


print(variance)
#find dispersion of z

49.1


### Task5
Omer trained a linear regression model and tested its performance on a test sample of 500 objects. <br>
On 400 of those, the model returned a prediction higher than expected by 0.5, and on the remaining 100, the model returned a prediction lower than expected by 0.7. What is the MSE for his model?
Limor claims that the linear regression model wasn't trained correctly, and we can do improve it by changing all the answers by a constant value. 
What will be her MSE?
You can assume that Limor found the smallest error under her constraints.
Return two values - Omer's and Limor's MSE.

In [42]:
n_high = 400
n_low = 100
n_objects = n_high+n_low
error_high = 0.5
error_low = -0.7

MSE = (n_high*error_high**2 + n_low*error_low**2)/n_objects
print(f'MSE_Omer = ',MSE)
change = -0.259
error_low+=change
error_high+=change
MSE = (n_high*error_high**2 + n_low*error_low**2)/n_objects
print(f'MSE_Limor = ',MSE)

MSE_Omer =  0.298
MSE_Limor =  0.230401
